In [6]:
import firebase_admin
from firebase_admin import credentials, firestore
import pandas as pd
import numpy as np
from scipy.sparse.linalg import svds
from keras.models import Model
from keras.layers import Input, Dense, Dropout, Concatenate
from keras.regularizers import l2
from keras.optimizers import Adam

In [7]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import db

# Khởi tạo kết nối với Firebase
cred = credentials.Certificate('serviceAccountKey.json')
firebase_admin.initialize_app(cred, {
    'databaseURL': 'https://coffee-4053c-default-rtdb.asia-southeast1.firebasedatabase.app/'
})

In [8]:
def GetEvaluate():
    evaluate_data = db.reference('DanhGia').get()

    evaluates = [
        {
            'MaKhachHang': evaluete['MaNguoiDung'],
            'MaSanPham': evaluete['MaSanPham'],
            'DiemDanhGia': evaluete['DiemDanhGia']
        }
        for evaluete in evaluate_data.values()
    ]

    return evaluates

In [9]:
evaluates = GetEvaluate()
evaluates

[{'MaKhachHang': 'KH0001', 'MaSanPham': 'SP0003', 'DiemDanhGia': 5},
 {'MaKhachHang': 'KH0001', 'MaSanPham': 'SP0004', 'DiemDanhGia': 4},
 {'MaKhachHang': 'KH0001', 'MaSanPham': 'SP0001', 'DiemDanhGia': 5},
 {'MaKhachHang': 'KH0001', 'MaSanPham': 'SP0001', 'DiemDanhGia': 5},
 {'MaKhachHang': 'KH0001', 'MaSanPham': 'SP0003', 'DiemDanhGia': 3},
 {'MaKhachHang': 'KH0001', 'MaSanPham': 'SP0002', 'DiemDanhGia': 3},
 {'MaKhachHang': 'KH0001', 'MaSanPham': 'SP0005', 'DiemDanhGia': 4},
 {'MaKhachHang': 'KH0001', 'MaSanPham': 'SP0004', 'DiemDanhGia': 4},
 {'MaKhachHang': 'KH0001', 'MaSanPham': 'SP0001', 'DiemDanhGia': 3},
 {'MaKhachHang': 'KH0001', 'MaSanPham': 'SP0002', 'DiemDanhGia': 1},
 {'MaKhachHang': 'KH0001', 'MaSanPham': 'SP0002', 'DiemDanhGia': 1},
 {'MaKhachHang': 'KH0001', 'MaSanPham': 'SP0001', 'DiemDanhGia': 3},
 {'MaKhachHang': 'KH0001', 'MaSanPham': 'SP0004', 'DiemDanhGia': 5},
 {'MaKhachHang': 'KH0001', 'MaSanPham': 'SP0004', 'DiemDanhGia': 5},
 {'MaKhachHang': 'KH0001', 'MaSanP

In [10]:
def GetBill():
    bill_data = db.reference('HoaDon').get()

    bills = []

    for bill in bill_data.values():
        user_id = bill['MaKhachHang']

        if user_id:
            details = bill['ChiTietHoaDon']

            for item in details.values():
                product_id = item['MaSanPham']
                quantity = item['SoLuong']

                bills.append({
                    "MaKhachHang": user_id,
                    "MaSanPham": product_id,
                    "SoLuong": quantity
                })

    return bills

In [11]:
bills = GetBill()
bills

[{'MaKhachHang': 'KH0001', 'MaSanPham': 'SP0002', 'SoLuong': 1},
 {'MaKhachHang': 'KH0001', 'MaSanPham': 'SP0005', 'SoLuong': 4},
 {'MaKhachHang': 'KH0001', 'MaSanPham': 'SP0007', 'SoLuong': 2},
 {'MaKhachHang': 'KH0001', 'MaSanPham': 'SP0004', 'SoLuong': 1},
 {'MaKhachHang': 'KH0001', 'MaSanPham': 'SP0002', 'SoLuong': 3},
 {'MaKhachHang': 'KH0001', 'MaSanPham': 'SP0004', 'SoLuong': 1}]

In [12]:
def create_matrices(bills, evaluates):
    bill_df = pd.DataFrame(bills)
    evaluate_df = pd.DataFrame(evaluates)

    bill_matrix = bill_df.pivot_table(index='MaKhachHang', columns='MaSanPham', values='SoLuong', fill_value=0)
    evaluate_matrix = evaluate_df.pivot_table(index='MaKhachHang', columns='MaSanPham', values='DiemDanhGia', fill_value=0)

    return bill_matrix, evaluate_matrix

In [13]:
bill_matrix, evaluate_matrix = create_matrices(bills, evaluates)

In [14]:
bill_matrix

MaSanPham,SP0002,SP0004,SP0005,SP0007
MaKhachHang,,,,
KH0001,2.0,1.0,4.0,2.0


In [15]:
evaluate_matrix

MaSanPham,SP0001,SP0002,SP0003,SP0004,SP0005,SP0006,SP0007,SP0008,SP0009,SP0010
MaKhachHang,,,,,,,,,,
KH0001,4.166667,3.166667,4.0,4.428571,2.75,0.00,0.0,0.0,0.000000,0.000
KH0002,2.875000,2.500000,2.0,2.000000,3.50,3.75,3.0,3.2,3.142857,2.875


In [16]:
weights = {"bill": 0.5, "evaluate": 1.0}

In [17]:
combined_matrix = bill_matrix * weights["bill"] + evaluate_matrix * weights["evaluate"]

In [18]:
combined_matrix

MaSanPham,SP0001,SP0002,SP0003,SP0004,SP0005,SP0006,SP0007,SP0008,SP0009,SP0010
MaKhachHang,,,,,,,,,,
KH0001,NaN,4.166667,NaN,4.928571,4.75,NaN,1.0,NaN,NaN,NaN
KH0002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
user_item_matrix = combined_matrix.fillna(0).values

In [20]:
user_item_matrix

array([[0.        , 4.16666667, 0.        , 4.92857143, 4.75      ,
        0.        , 1.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ]])

In [21]:
# Bước 1: Áp dụng SVD
k_max = min(user_item_matrix.shape) -1
k = min(50, k_max)

u, sigma, vt = svds(user_item_matrix, k=k)
sigma_matrix = np.diag(sigma)
user_factors = np.dot(u, sigma_matrix)
item_factors = np.dot(sigma_matrix, vt).T

In [22]:
# Bước 2: Xây dựng Mạng Nơ-ron cải tiến
user_input = Input(shape=(user_factors.shape[1],), name='user_input')
item_input = Input(shape=(item_factors.shape[1],), name='item_input')

user_hidden = Dense(128, activation='relu', kernel_regularizer=l2(0.01))(user_input)
user_hidden = Dropout(0.3)(user_hidden)
user_hidden = Dense(64, activation='relu', kernel_regularizer=l2(0.01))(user_hidden)

item_hidden = Dense(128, activation='relu', kernel_regularizer=l2(0.01))(item_input)
item_hidden = Dropout(0.3)(item_hidden)
item_hidden = Dense(64, activation='relu', kernel_regularizer=l2(0.01))(item_hidden)

combined = Concatenate()([user_hidden, item_hidden])

hidden = Dense(64, activation='relu')(combined)
hidden = Dropout(0.3)(hidden)
hidden = Dense(32, activation='relu')(hidden)
hidden = Dropout(0.3)(hidden)
hidden = Dense(16, activation='relu')(hidden)
output = Dense(1, activation='linear')(hidden)

model = Model(inputs=[user_input, item_input], outputs=output)
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

In [23]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ user_input (InputLayer)   │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ item_input (InputLayer)   │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 128)            │            256 │ user_input[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 128)            │            256 │ item_input[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout (Dropout)         │ (None, 128)            │              0 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_1 (Dropout)       │ (None, 128)            │              0 │ dense_2[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 64)             │          8,256 │ dropout[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_3 (Dense)           │ (None, 64)             │          8,256 │ dropout_1[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 128)            │              0 │ dense_1[0][0],         │
│                           │                        │                │ dense_3[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_4 (Dense)           │ (None, 64)             │          8,256 │ concatenate[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_2 (Dropout)       │ (None, 64)             │              0 │ dense_4[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_5 (Dense)           │ (None, 32)             │          2,080 │ dropout_2[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_3 (Dropout)       │ (None, 32)             │              0 │ dense_5[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_6 (Dense)           │ (None, 16)             │            528 │ dropout_3[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_7 (Dense)           │ (None, 1)              │             17 │ dense_6[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 27,905 (109.00 KB)

 Trainable params: 27,905 (109.00 KB)

 Non-trainable params: 0 (0.00 B)

In [24]:
# Chuẩn bị dữ liệu đầu vào
user_ids, item_ids = user_item_matrix.nonzero()
train_user_factors = user_factors[user_ids]
train_item_factors = item_factors[item_ids]
train_ratings = user_item_matrix[user_ids, item_ids]

In [25]:
# Bước 3: Huấn luyện mô hình với Dropout
model.fit(
    [train_user_factors, train_item_factors],
    train_ratings,
    epochs=100,
    batch_size=64,
    validation_split=0.2,
    shuffle=True
)

Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - loss: 18.2735 - val_loss: 2.4369
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 318ms/step - loss: 20.6683 - val_loss: 2.1741
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - loss: 15.3670 - val_loss: 1.9533
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: 18.3874 - val_loss: 1.8043
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 13.3738 - val_loss: 1.7060
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - loss: 14.6361 - val_loss: 1.6532
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 9.6609 - val_loss: 1.6403
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: 11.1634 - val_loss: 1.6691
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - loss: 12.8542 - val_loss: 1.7383
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - loss: 10.1980 - val_loss: 1.8573
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 8.8748 - val_loss: 2.0392
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 8.3928 

In [26]:
user_factors

array([[8.07554502],
       [0.        ]])

In [27]:
def get_top_n_recommendations(user_id, all_user_ids, all_product_ids, user_factors, item_factors, model, top_n=5):
    if user_id not in all_user_ids:
        print(f"User {user_id} not found in the data.")
        return []

    # Get the index of the user in the user factors matrix
    user_index = all_user_ids.index(user_id)
    user_vector = user_factors[user_index].reshape(1, -1)  # Reshape to 2D

    # Predict ratings for all products and keep only the top-N
    product_ratings = []
    for product_index, product_id in enumerate(all_product_ids):
        # Skip products the user has already interacted with
        if user_item_matrix[user_index, product_index] > 0:
            continue

        # Get the item factor vector and predict the rating
        product_vector = item_factors[product_index].reshape(1, -1)  # Reshape to 2D
        predicted_rating = model.predict([user_vector, product_vector])[0][0]

        # Collect the product ID and predicted rating
        product_ratings.append((product_id, predicted_rating))

    # Sort by predicted rating in descending order and select the top-N products
    product_ratings.sort(key=lambda x: x[1], reverse=True)
    top_recommendations = [product_id for product_id, rating in product_ratings[:top_n]]

    return top_recommendations

In [28]:
user_id = 'KH0001'
top_n = 5
all_user_ids = list(bill_matrix.index)
all_product_ids = list(bill_matrix.columns)

recommendations = get_top_n_recommendations(user_id, all_user_ids, all_product_ids, user_factors, item_factors, model, top_n)
recommendations

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step


['SP0002', 'SP0005']

In [29]:
model.save("recommendation_model.h5")

In [30]:
import numpy as np
import pickle

# Lưu user_factors, item_factors và ánh xạ ID khách hàng, sản phẩm
np.save("user_factors.npy", user_factors)
np.save("item_factors.npy", item_factors)

# Tạo và lưu ánh xạ ID
with open("user_index_mapping.pkl", "wb") as f1, open("item_index_mapping.pkl", "wb") as f2:
    pickle.dump({user_id: index for index, user_id in enumerate(bill_matrix.index)}, f1)
    pickle.dump({item_id: index for index, item_id in enumerate(bill_matrix.columns)}, f2)
